In [ ]:
import requests
import pandas as pd
from sklearn.cluster import KMeans

API_KEY = "TsFor00einWlC1GYbiF8KimOG-1XOIHI-DrbB55NLxQ"

addresses = [
    # Bengaluru
    "Vidhan Soudha, Bengaluru, Karnataka",
    "Cubbon Park, Bengaluru, Karnataka",
    "Lalbagh Botanical Garden, Bengaluru, Karnataka",
    "Bangalore Palace, Bengaluru, Karnataka",
    "ISKCON Temple, Bengaluru, Karnataka",
    "Bannerghatta Biological Park, Bengaluru, Karnataka",
    "Innovative Film City, Bengaluru, Karnataka",
    "Nandi Hills, Chikkaballapur, Karnataka",

    # Mysuru
    "Mysore Palace, Mysuru, Karnataka",
    "Chamundi Hill, Mysuru, Karnataka",
    "Brindavan Gardens, Mysuru, Karnataka",
    "St. Philomena's Church, Mysuru, Karnataka",

    # Mangaluru
    "Mangaladevi Temple, Mangaluru, Karnataka",
    "Panambur Beach, Mangaluru, Karnataka",
    "Kadri Manjunath Temple, Mangaluru, Karnataka",

    # Chikmagalur
    "Mullayanagiri Peak, Chikmagalur, Karnataka",
    "Coffee Museum, Chikmagalur, Karnataka",
    "Hebbe Falls, Chikmagalur, Karnataka",
    "Kudremukh National Park, Chikmagalur, Karnataka",

    # Shivamogga
    "Jog Falls, Shivamogga, Karnataka",
    "Tyavarekoppa Lion Safari, Shivamogga, Karnataka",
    "Mandagadde Bird Sanctuary, Shivamogga, Karnataka",

    # Hampi
    "Virupaksha Temple, Hampi, Karnataka",
    "Vijaya Vittala Temple, Hampi, Karnataka",
    "Lotus Mahal, Hampi, Karnataka",
    "Elephant Stables, Hampi, Karnataka",

    # Coorg (Kodagu)
    "Abbey Falls, Madikeri, Karnataka",
    "Raja's Seat, Madikeri, Karnataka",
    "Dubare Elephant Camp, Coorg, Karnataka",
    "Talakaveri, Coorg, Karnataka",

    # Belagavi
    "Belgaum Fort, Belagavi, Karnataka",
    "Kapileshwara Temple, Belagavi, Karnataka",
    "Gokak Falls, Belagavi, Karnataka",

    # Hubballi-Dharwad
    "Unkal Lake, Hubballi, Karnataka",
    "Indira Gandhi Glass House Garden, Hubballi, Karnataka",
    "Banashankari Temple, Hubballi, Karnataka",

    # Bijapur (Vijayapura)
    "Gol Gumbaz, Vijayapura, Karnataka",
    "Ibrahim Rauza, Vijayapura, Karnataka",
    "Malik-e-Maidan, Vijayapura, Karnataka",

    # Udupi
    "Sri Krishna Temple, Udupi, Karnataka",
    "Malpe Beach, Udupi, Karnataka",
    "Kapu Lighthouse, Udupi, Karnataka",

    # Gokarna
    "Om Beach, Gokarna, Karnataka",
    "Kudle Beach, Gokarna, Karnataka",
    "Mahabaleshwar Temple, Gokarna, Karnataka",

    # Davangere
    "Kunduvada Kere, Davangere, Karnataka",
    "Bathi Gudda, Davangere, Karnataka",

    # Badami, Pattadakal, and Aihole
    "Badami Caves, Badami, Karnataka",
    "Pattadakal Temples, Pattadakal, Karnataka",
    "Aihole Temples, Aihole, Karnataka",

    # Ballari
    "Bellary Fort, Ballari, Karnataka",
    "Daroji Sloth Bear Sanctuary, Ballari, Karnataka",

    # Other Key Locations
    "Ranganathittu Bird Sanctuary, Mandya, Karnataka",
    "Shivanasamudra Falls, Mandya, Karnataka",
    "Bhadra Wildlife Sanctuary, Chikkamagaluru, Karnataka",
    "Banavasi Temple, Uttara Kannada, Karnataka"
]

# ----------------------------------------------
# 2. Geocoding Function
# ----------------------------------------------
def geocode_address(address):
    geocode_url = "https://geocode.search.hereapi.com/v1/geocode"
    params = {
        "q": address,
        "apiKey": API_KEY
    }

    response = requests.get(geocode_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["items"]:
            position = data["items"][0]["position"]
            lat, lng = position["lat"], position["lng"]
            locality = data["items"][0]["address"].get("label", "Unknown locality")
            return lat, lng, locality
        else:
            print(f"No geocoding results for: {address}")
            return None, None, None
    else:
        print("Geocoding API error:", response.status_code, response.reason)
        return None, None, None


def fetch_amenities_data(lat, lng):

    import random
    return {
        "cafes": random.randint(0, 20),
        "hospitals": random.randint(0, 10),
        "parks": random.randint(0, 15)
    }

data_rows = []
for addr in addresses:
    lat, lng, locality = geocode_address(addr)
    if lat is not None and lng is not None:
        amenities = fetch_amenities_data(lat, lng)
        data_rows.append({
            "address": addr,
            "locality": locality,
            "latitude": lat,
            "longitude": lng,
            "cafes": amenities["cafes"],
            "hospitals": amenities["hospitals"],
            "parks": amenities["parks"]
        })

# Create a DataFrame of our training data
df = pd.DataFrame(data_rows)
print("Training DataFrame:\n", df)

# K Means
X_coords = df[["latitude", "longitude"]].values
kmeans_coords = KMeans(n_clusters=3, random_state=42)
kmeans_coords.fit(X_coords)
df["coordinate_cluster"] = kmeans_coords.labels_

# Clustering based on amenities
X_amenities = df[["cafes", "hospitals", "parks"]].values
kmeans_amenities = KMeans(n_clusters=3, random_state=42)
kmeans_amenities.fit(X_amenities)
df["amenities_cluster"] = kmeans_amenities.labels_

# Map amenities clusters to human-readable labels
amenities_label_map = {
    0: "good amenities",
    1: "limited amenities",
    2: "no amenities"
}
df["amenities_label"] = df["amenities_cluster"].map(amenities_label_map)

# Map coordinate clusters to user-friendly descriptions
coordinate_label_map = {
    0: "Central Karnataka",
    1: "Southern Karnataka",
    2: "Coastal Karnataka"
}
df["coordinate_label"] = df["coordinate_cluster"].map(coordinate_label_map)

print("\nClusters based on coordinates and amenities:\n", df)


def predict_cluster_for_new_address(new_address):
    """
    Predict both coordinate and amenities clusters for a new address,
    and provide meaningful descriptions for the clusters.
    """
    lat, lng, locality = geocode_address(new_address)
    if lat is not None and lng is not None:
        amenities = fetch_amenities_data(lat, lng)
        coord_features = [[lat, lng]]
        amenity_features = [[amenities["cafes"], amenities["hospitals"], amenities["parks"]]]

        coord_cluster = kmeans_coords.predict(coord_features)[0]
        amenity_cluster = kmeans_amenities.predict(amenity_features)[0]

        return {
            "locality": locality,
            "coordinate_cluster": coordinate_label_map[coord_cluster],
            "amenities_cluster": amenities_label_map[amenity_cluster],
            "amenities": amenities
        }
    else:
        return None



Geocoding API error: 429 Too Many Requests
Training DataFrame:
                                               address  \
0                 Vidhan Soudha, Bengaluru, Karnataka   
1                   Cubbon Park, Bengaluru, Karnataka   
2      Lalbagh Botanical Garden, Bengaluru, Karnataka   
3              Bangalore Palace, Bengaluru, Karnataka   
4                 ISKCON Temple, Bengaluru, Karnataka   
5   Bannerghatta Biological Park, Bengaluru, Karna...   
6          Innovative Film City, Bengaluru, Karnataka   
7              Nandi Hills, Chikkaballapur, Karnataka   
8                    Mysore Palace, Mysuru, Karnataka   
9                    Chamundi Hill, Mysuru, Karnataka   
10               Brindavan Gardens, Mysuru, Karnataka   
11          St. Philomena's Church, Mysuru, Karnataka   
12           Mangaladevi Temple, Mangaluru, Karnataka   
13               Panambur Beach, Mangaluru, Karnataka   
14       Kadri Manjunath Temple, Mangaluru, Karnataka   
15         Mullayanagiri

In [ ]:
# Example: Predict for a new address/pincode
new_locality = input("Enter a locality or pincode: ")
predicted_clusters = predict_cluster_for_new_address(new_locality)

if predicted_clusters is not None:
    print(f"\nNew address '{new_locality}' is located in '{predicted_clusters['locality']}'.")
    print(f"Geographic Area: {predicted_clusters['coordinate_cluster']}")
    print(f"Amenities Status: {predicted_clusters['amenities_cluster']}")
    print("Amenities Details:")
    print(f"  Cafes: {predicted_clusters['amenities']['cafes']}")
    print(f"  Hospitals: {predicted_clusters['amenities']['hospitals']}")
    print(f"  Parks: {predicted_clusters['amenities']['parks']}")
else:
    print("Unable to classify the new address due to geocoding failure.")


Enter a locality or pincode: 560029

New address '560029' is located in '560029, Bengaluru, Karnataka, India'.
Geographic Area: Coastal Karnataka
Amenities Status: good amenities
Amenities Details:
  Cafes: 3
  Hospitals: 2
  Parks: 13
